In [ ]:
# Gerekli kütüphaneleri yükleyelim
!pip install ultralytics torchmetrics -q

import os
import yaml
import random
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

os.system('pip install pycocotools')

# Gerekli importları tazeleyelim
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import torch
from PIL import Image

# PyTorch ve Model Kütüphaneleri
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, ssdlite320_mobilenet_v3_large, retinanet_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.retinanet import RetinaNetHead
from torch.optim import SGD
from ultralytics import YOLO

# Metrikler
try:
    from torchmetrics.detection.mean_ap import MeanAveragePrecision
except ImportError:
    print("Torchmetrics yüklenemedi, metrik hesaplaması çalışmayabilir.")

# Cihaz ayarı (GPU varsa kullan)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"✅ Çalışma ortamı hazır. Cihaz: {DEVICE}")

In [ ]:
# --- AYARLAR ---
EPOCHS =5       # Ödev isterine göre 100 olmalı (Test için 5-10 yapabilirsin)
BATCH_SIZE = 4        # GPU hafızası yetmezse 2'ye düşür
IMG_SIZE = 320      # Görüntü boyutu
NUM_CLASSES_PT = 2    # PyTorch için: 0=Background, 1=Person

# --- VERİ YOLU BULMA ---
start_dir = "/kaggle/input/thermal-image-people-detection/People Detection - Themalt"
root_dir = "/kaggle/input/thermal-image-people-detection/People Detection - Themal"

print("🕵️‍♂️ Veri seti aranıyor...")
for dirpath, dirnames, filenames in os.walk(start_dir):
    if "train" in dirnames and "valid" in dirnames:
        root_dir = dirpath
        break

if root_dir:
    train_path = os.path.join(root_dir, "train")
    valid_path = os.path.join(root_dir, "valid")
    print(f"✅ Veri seti bulundu: {root_dir}")
else:
    print("❌ HATA: Veri seti bulunamadı! Lütfen Kaggle sağ menüden veri setini eklediğine emin ol.")

# --- YOLO İÇİN YAML OLUŞTURMA ---
if root_dir:
    data_yaml = {
        'train': os.path.join(train_path, "images"),
        'val': os.path.join(valid_path, "images"),
        'nc': 1,
        'names': ['person']
    }
    with open('dataset.yaml', 'w') as f:
        yaml.dump(data_yaml, f)
    print("✅ YOLO için 'dataset.yaml' oluşturuldu.")

In [ ]:
class ThermalDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.image_dir = os.path.join(root_dir, "images")
        self.label_dir = os.path.join(root_dir, "labels")
        self.imgs = sorted([f for f in os.listdir(self.image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = os.path.join(self.image_dir, img_name)
        label_path = os.path.join(self.label_dir, os.path.splitext(img_name)[0] + ".txt")

        # Resmi oku ve RGB yap
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape
        img_pil = Image.fromarray(img)

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    if len(parts) == 5:
                        cls, cx, cy, bw, bh = parts
                        # YOLO formatından (normalize) -> Pascal VOC formatına (piksel) çevir
                        x_min = (cx - bw/2) * w
                        y_min = (cy - bh/2) * h
                        x_max = (cx + bw/2) * w
                        y_max = (cy + bh/2) * h

                        # Hatalı kutuları engelle
                        if x_max > x_min and y_max > y_min:
                            boxes.append([x_min, y_min, x_max, y_max])
                            labels.append(1) # 1: Person

        # Tensörlere çevir
        target = {}
        if len(boxes) > 0:
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["area"] = (target["boxes"][:, 3] - target["boxes"][:, 1]) * (target["boxes"][:, 2] - target["boxes"][:, 0])
            target["iscrowd"] = torch.zeros((len(labels),), dtype=torch.int64)
            target["image_id"] = torch.tensor([idx])
        else:
            # Kutu yoksa boş tensör gönder
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
            target["area"] = torch.zeros((0,), dtype=torch.float32)
            target["iscrowd"] = torch.zeros((0,), dtype=torch.int64)
            target["image_id"] = torch.tensor([idx])

        if self.transforms:
            from torchvision import transforms as T
            # Sadece ToTensor uyguluyoruz, gerekirse augmentasyon eklenebilir
            t = T.Compose([T.ToTensor()])
            img_tensor = t(img_pil)
        
        return img_tensor, target

    def __len__(self):
        return len(self.imgs)

def collate_fn(batch):
    return tuple(zip(*batch))

# Datasetleri Oluştur
train_dataset = ThermalDataset(train_path, transforms=True)
valid_dataset = ThermalDataset(valid_path, transforms=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn, num_workers=2)

print(f"✅ Dataset Hazır. Train: {len(train_dataset)}, Valid: {len(valid_dataset)} görsel.")

In [ ]:
def visualize_sample(loader):
    images, targets = next(iter(loader))
    img = images[0].permute(1, 2, 0).cpu().numpy().copy()
    boxes = targets[0]['boxes'].cpu().numpy()
    
    plt.figure(figsize=(10,8))
    for box in boxes:
        x_min, y_min, x_max, y_max = box.astype(int)
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    plt.imshow(img)
    plt.title("Örnek Veri Kontrolü")
    plt.axis('off')
    plt.show()

try:
    visualize_sample(train_loader)
except Exception as e:
    print(f"Görselleştirme hatası: {e}")

In [ ]:
def train_model(model, loader, epochs, name):
    print(f"\n🚀 [{name}] Eğitimi Başlıyor... ({epochs} Epoch)")
    model.to(DEVICE)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
    
    model.train()
    loss_history = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        batch_count = 0
        
        for images, targets in loader:
            images = [img.to(DEVICE) for img in images]
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
            
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            
            epoch_loss += losses.item()
            batch_count += 1
        
        avg_loss = epoch_loss/batch_count
        loss_history.append(avg_loss)
        
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"   Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f}")
            
    print(f"✅ {name} Eğitimi Tamamlandı.")
    return model, loss_history

In [ ]:
print("=== MODEL 1: YOLOv8m Eğitimi ===")

model_yolo = YOLO('yolov8m.pt') 

results_yolo = model_yolo.train(
    data='dataset.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=8, # YOLO genelde daha optimize çalışır, batch'i artırabilirsin
    name='yolo_thermal_project',
    verbose=False
)

print("YOLO eğitimi bitti.")

In [ ]:
print("=== MODEL 2: Faster R-CNN (ResNet-50) Eğitimi ===")

# Modeli Yükle
model_frcnn = fasterrcnn_resnet50_fpn_v2(weights="DEFAULT")

# Çıkış katmanını (Head) değiştir (2 Sınıf: Background + Person)
in_features = model_frcnn.roi_heads.box_predictor.cls_score.in_features
model_frcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES_PT)

# Eğit
trained_frcnn, loss_frcnn = train_model(model_frcnn, train_loader, EPOCHS, "Faster R-CNN")

In [ ]:
print("=== MODEL 3: RetinaNet Eğitimi ===")

# Modeli Yükle
model_retina = retinanet_resnet50_fpn_v2(weights="DEFAULT")

# Head katmanını değiştir
num_anchors = model_retina.head.classification_head.num_anchors
model_retina.head = RetinaNetHead(in_channels=256, num_anchors=num_anchors, num_classes=NUM_CLASSES_PT)

# Eğit (Eğer CUDA hatası alırsan batch size'ı düşür)
trained_retina, loss_retina = train_model(model_retina, train_loader, EPOCHS, "RetinaNet")

In [ ]:
print("=== MODEL 4: SSDLite (MobileNetV3) Eğitimi ===")

# 1. Pretrained ağırlıkları al
base_model = ssdlite320_mobilenet_v3_large(weights="DEFAULT")
pretrained_dict = base_model.state_dict()

# 2. Yeni boş model oluştur (Bizim sınıf sayımızla)
model_ssd = ssdlite320_mobilenet_v3_large(weights=None, num_classes=NUM_CLASSES_PT)
model_dict = model_ssd.state_dict()

# 3. Sadece uyuşan katmanları kopyala (Head hariç)
filtered_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
model_dict.update(filtered_dict)
model_ssd.load_state_dict(model_dict)

print("✅ SSDLite ağırlıkları başarıyla aktarıldı.")

# Eğit
trained_ssd, loss_ssd = train_model(model_ssd, train_loader, EPOCHS, "SSDLite")

In [ ]:
def evaluate_pytorch(model, loader, device):
    print(f"⚙️ Değerlendirme yapılıyor... (Cihaz: {device})")
    model.eval()
    metric = MeanAveragePrecision(iou_type="bbox")
    
    # İlerleme durumunu görmek için basit sayaç
    batch_index = 0
    total_batches = len(loader)
    
    with torch.no_grad():
        for images, targets in loader:
            images = [img.to(device) for img in images]
            preds = model(images)
            
            # CPU'ya alıp formata uygun hale getir
            preds_cpu = [{k: v.to("cpu") for k, v in p.items()} for p in preds]
            targets_cpu = [{k: v.to("cpu") for k, v in t.items()} for t in targets]
            
            metric.update(preds_cpu, targets_cpu)
            
            batch_index += 1
            if batch_index % 10 == 0:
                print(f"   Batch {batch_index}/{total_batches} işlendi.")
            
    print("📈 Metrikler hesaplanıyor...")
    result = metric.compute()
    return result['map_50'].item(), result['map'].item()

# --- FİNAL HESAPLAMA ---
print("\n📊 --- FİNAL SONUÇLARI HESAPLANIYOR ---")

# 1. YOLO Sonuçları (Zaten hazırdı)
# Not: Eğer results_yolo nesnesi tensör ise .item() gerekebilir, değilse direkt alınır.
try:
    yolo_map50 = results_yolo.box.map50
    yolo_map = results_yolo.box.map
except:
    yolo_map50 = 0.0
    yolo_map = 0.0

# 2. PyTorch Modelleri
print("\n--- Faster R-CNN Değerlendiriliyor ---")
frcnn_map50, frcnn_map = evaluate_pytorch(trained_frcnn, valid_loader, DEVICE)

print("\n--- RetinaNet Değerlendiriliyor ---")
retina_map50, retina_map = evaluate_pytorch(trained_retina, valid_loader, DEVICE)

print("\n--- SSDLite Değerlendiriliyor ---")
ssd_map50, ssd_map = evaluate_pytorch(trained_ssd, valid_loader, DEVICE)

# Tabloyu Yazdır
print("\n" + "="*65)
print(f"| {'MODEL ADI':<20} | {'mAP@0.50 (Doğruluk)':<20} | {'mAP@0.50:0.95':<15} |")
print(f"|{'-'*22}|{'-'*22}|{'-'*17}|")
print(f"| {'YOLOv8m':<20} | {yolo_map50:.4f}               | {yolo_map:.4f}          |")
print(f"| {'Faster R-CNN':<20} | {frcnn_map50:.4f}               | {frcnn_map:.4f}          |")
print(f"| {'RetinaNet':<20} | {retina_map50:.4f}               | {retina_map:.4f}          |")
print(f"| {'SSDLite':<20} | {ssd_map50:.4f}               | {ssd_map:.4f}          |")
print("="*65)

In [1]:
import torch
import gc

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()
gc.collect()



0

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import retinanet_resnet50_fpn

num_classes = 2  # arka plan + 1 sınıf (örnek)

model_frcnn = fasterrcnn_resnet50_fpn(num_classes=num_classes)
model_retina = retinanet_resnet50_fpn(num_classes=num_classes)

model_frcnn.to(DEVICE)
model_retina.to(DEVICE)



In [ ]:
params = list(model_frcnn.parameters()) + list(model_retina.parameters())

optimizer = torch.optim.SGD(
    params,
    lr=0.0005,
    momentum=0.9,
    weight_decay=0.0005
)


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=1,          # 🔴 RAM koruma
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0
)


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    model_frcnn.train()
    model_retina.train()

    total_loss_epoch = 0

    for images, targets in train_loader:
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        # 🔹 Faster R-CNN loss
        loss_dict_frcnn = model_frcnn(images, targets)
        loss_frcnn = sum(loss for loss in loss_dict_frcnn.values())

        # 🔹 RetinaNet loss
        loss_dict_retina = model_retina(images, targets)
        loss_retina = sum(loss for loss in loss_dict_retina.values())

        # 🔥 ORTAK LOSS
        total_loss = loss_frcnn + loss_retina

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_loss_epoch += total_loss.item()

    print(
        f"[Epoch {epoch+1}/{EPOCHS}] "
        f"FRCNN Loss: {loss_frcnn.item():.4f} | "
        f"Retina Loss: {loss_retina.item():.4f}"
    )





In [ ]:
# =====================================
# FINE-TUNING (DERİN ÖĞRENME - GÜVENLİ MOD)
# =====================================
import gc

print("🔧 Fine-Tuning Başladı (Faster R-CNN)")

# 1️⃣ Bellek Temizliği: Önceki hatalardan kalan RAM yükünü boşaltalım
torch.cuda.empty_cache()
gc.collect()

# 2️⃣ Veri Yükleyiciyi Bellek Dostu Yapalım (num_workers=0 hatayı çözer)
train_loader_fine = DataLoader(
    train_dataset, 
    batch_size=2,          # RAM kullanımı için batch'i 2'ye düşürdük
    shuffle=True, 
    collate_fn=collate_fn, 
    num_workers=0          # "Killed" hatasını engelleyen kritik ayar
)

# 3️⃣ Katman Ayarları
# Backbone'u dondur (Genel özellikleri koru)
for param in trained_frcnn.backbone.parameters():
    param.requires_grad = False

# Sadece son katman bloğunu (layer4) ve kafayı (head) eğitmek istersen (daha derin ince ayar):
for param in trained_frcnn.backbone.body.layer4.parameters():
    param.requires_grad = True

# 4️⃣ Optimizer ve Parametreler
trainable_params = [p for p in trained_frcnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    trainable_params,
    lr=0.0001,      # Çok küçük bir LR: Mevcut bilgileri bozmadan üzerine ekler
    momentum=0.9,
    weight_decay=0.0005
)

EPOCHS_FINE = 15
trained_frcnn.to(DEVICE)
trained_frcnn.train()

# 5️⃣ Eğitim Döngüsü
try:
    for epoch in range(EPOCHS_FINE):
        total_loss = 0
        batch_count = 0

        for images, targets in train_loader_fine:
            images = [img.to(DEVICE) for img in images]
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

            loss_dict = trained_frcnn(images, targets)
            loss = sum(l for l in loss_dict.values())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

        print(f"[Fine-Tuning] Epoch {epoch+1}/{EPOCHS_FINE} | Ortalama Loss: {total_loss/batch_count:.4f}")
        
except Exception as e:
    print(f"⚠️ Eğitim sırasında bir hata oluştu: {e}")
    print("İpucu: Eğer hala 'Killed' alıyorsan batch_size=1 yapabilirsin.")

print("✅ Fine-Tuning tamamlandı.")




def ensemble_detector(models, image, device, iou_thr=0.5):
    boxes_all, scores_all, model_index = [], [], []

    for i, model in enumerate(models):
        model.to(device) # Modelin GPU'da olduğundan emin ol
        model.eval()
        with torch.no_grad():
            # [image] formatı PyTorch modelleri için doğrudur (0-1 arası float tensor)
            output = model([image.to(device)])[0]

        # Sadece belirli bir güven eşiğinin üzerindeki kutuları alalım (Örn: 0.5)
        # Bu, baskınlık analizinin daha anlamlı olmasını sağlar.
        keep = output["scores"] > 0.4 
        
        if len(output["boxes"][keep]) > 0:
            boxes_all.append(output["boxes"][keep].cpu())
            scores_all.append(output["scores"][keep].cpu())
            model_index += [i] * len(output["boxes"][keep])

    if len(boxes_all) == 0:
        return torch.tensor([]), torch.tensor([]), []

    boxes_all = torch.cat(boxes_all)
    scores_all = torch.cat(scores_all)

    selected = []
    used = set()

    # Skorlara göre sıralayalım ki en iyi kutudan başlayalım (NMS mantığı)
    indices = torch.argsort(scores_all, descending=True)

    for i in indices:
        idx = i.item()
        if idx in used:
            continue

        overlaps = box_iou(boxes_all[idx].unsqueeze(0), boxes_all)[0]
        group = torch.where(overlaps > iou_thr)[0]

        selected.append(idx)
        used.update(group.tolist())

    return boxes_all[selected], scores_all[selected], [model_index[i] for i in selected]




In [ ]:
# =====================================
# MODEL DOMINANCE ANALYSIS
# =====================================

models = [trained_frcnn, trained_retina, trained_ssd]
model_names = ["Faster R-CNN", "RetinaNet", "SSDLite"]

dominance = {name: 0 for name in model_names}

for images, _ in valid_loader:
    image = images[0]
    _, _, model_ids = ensemble_detector(models, image, DEVICE)

    for mid in model_ids:
        dominance[model_names[mid]] += 1

print("\n📊 MODEL BASKINLIK ANALİZİ")
print("="*35)
for k, v in dominance.items():
    print(f"{k:<15}: {v}")
